<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/dreamstack_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
# dreamstack.py
# A layered simulation where fields spawn structures, structures spawn agents,
# agents dream futures, and dreams feed back into evolving physics.

import os
import sys
import math
import time
import json
import csv
import argparse
from dataclasses import dataclass, asdict
from typing import Tuple, List, Dict, Optional

import numpy as np
import scipy.ndimage as nd
import matplotlib.pyplot as plt

# =========================
# Config
# =========================
@dataclass
class Config:
    # Grid/time
    nx: int = 192
    ny: int = 192
    dx: float = 1.0
    dt: float = 0.08
    steps: int = 1500

    # φ^4 physics
    c: float = 1.0
    lam: float = 1.0
    v: float = 1.0
    eta: float = 0.01

    # Initialization
    init: str = "random"       # random|blob|bias
    random_amp: float = 0.02
    blob_sigma: float = 18.0
    bias_value: float = 0.9
    seed: int = 42
    dtype: str = "float32"     # float32|float64

    # Structure detection
    struct_sigma: float = 1.0
    struct_win: int = 5
    struct_thresh: float = 0.05

    # Patch genes (per-cell adaptation)
    genes_mutation_rate: float = 0.002
    genes_lambda_range: Tuple[float, float] = (0.3, 2.0)
    genes_noise_range: Tuple[float, float] = (0.0, 0.05)
    genes_damp_range: Tuple[float, float] = (0.90, 1.10)

    # Perturbations
    perturb_mag: float = 0.006
    perturb_prob: float = 0.001

    # SubFields (agents)
    n_subfields: int = 24
    sub_size: int = 15
    plan_steps: int = 12
    actions_per_step: int = 3
    action_kick: float = 0.03

    # Dream levels and selection
    dream_levels: int = 2
    survivors_per_level: int = 2

    # I/O
    out_dir: str = "out_dreamstack"
    save_every: int = 50
    cmap: str = "seismic"

# =========================
# Utilities
# =========================
def ensure_dir(p: str):
    os.makedirs(p, exist_ok=True)

def save_json(path: str, obj):
    with open(path, "w", encoding="utf-8") as f:
        json.dump(obj, f, indent=2)

def laplacian_5pt(a: np.ndarray, dx: float) -> np.ndarray:
    return (np.roll(a,1,0)+np.roll(a,-1,0)+np.roll(a,1,1)+np.roll(a,-1,1)-4*a)/(dx*dx)

def gradients_central(a: np.ndarray, dx: float):
    gx = (np.roll(a,-1,0) - np.roll(a,1,0))/(2*dx)
    gy = (np.roll(a,-1,1) - np.roll(a,1,1))/(2*dx)
    return gx, gy

def V_phi4(phi: np.ndarray, lam: float, v: float):
    return 0.25 * lam * (phi*phi - v*v)**2

def dV_dphi(phi: np.ndarray, lam: float, v: float):
    return lam * phi * (phi*phi - v*v)

def energy_density(phi, phi_prev, dt, dx, c, lam, v):
    vel = (phi - phi_prev) / dt
    Ek = 0.5 * (vel*vel)
    gx, gy = gradients_central(phi, dx)
    Eg = 0.5 * (c*c) * (gx*gx + gy*gy)
    Ep = V_phi4(phi, lam, v)
    return Ek + Eg + Ep

def total_energy(phi, phi_prev, dt, dx, c, lam, v):
    ed = energy_density(phi, phi_prev, dt, dx, c, lam, v)
    return float(ed.sum() * dx * dx)

# =========================
# Structure detection
# =========================
def detect_structures(phi: np.ndarray, dx: float, lam: float, v: float,
                      sigma=1.0, win=5, thresh=0.05):
    E = 0.5*(np.gradient(phi, dx, axis=0)**2 + np.gradient(phi, dx, axis=1)**2) + V_phi4(phi, lam, v)
    Es = nd.gaussian_filter(E, sigma=sigma)
    local_max = (Es == nd.maximum_filter(Es, size=win))
    sites = np.argwhere(local_max & (Es > thresh))
    return sites, Es

# =========================
# Patch genes
# =========================
class PatchGenes:
    def __init__(self, lam, noise_amp, damping, cfg: Config):
        self.lam = lam
        self.noise_amp = noise_amp
        self.damping = damping
        self.cfg = cfg

    @staticmethod
    def random(cfg: Config, rng):
        lam = rng.uniform(*cfg.genes_lambda_range)
        noise = rng.uniform(*cfg.genes_noise_range)
        damp = rng.uniform(*cfg.genes_damp_range)
        return PatchGenes(lam, noise, damp, cfg)

    def mutate(self, rng):
        if rng.random() < self.cfg.genes_mutation_rate:
            self.lam = np.clip(self.lam + rng.normal(0, 0.1), *self.cfg.genes_lambda_range)
        if rng.random() < self.cfg.genes_mutation_rate:
            self.noise_amp = np.clip(self.noise_amp + rng.normal(0, 0.01), *self.cfg.genes_noise_range)
        if rng.random() < self.cfg.genes_mutation_rate:
            self.damping = np.clip(self.damping + rng.normal(0, 0.01), *self.cfg.genes_damp_range)

# =========================
# Field simulator with per-cell genes
# =========================
class Field:
    def __init__(self, cfg: Config, rng: np.random.Generator):
        self.cfg = cfg
        self.rng = rng
        self.nx, self.ny = cfg.nx, cfg.ny
        self.dx, self.dt = cfg.dx, cfg.dt
        self.c, self.v = cfg.c, cfg.v
        self.eta = cfg.eta
        self.dtype = np.float64 if cfg.dtype == "float64" else np.float32

        self.phi = np.zeros((self.nx, self.ny), dtype=self.dtype)
        self.phi_prev = np.zeros_like(self.phi)

        # genes per cell
        self.genes = np.empty((self.nx, self.ny), dtype=object)
        for i in range(self.nx):
            for j in range(self.ny):
                self.genes[i,j] = PatchGenes.random(cfg, rng)

        self._initialize(cfg.init, cfg.random_amp, cfg.blob_sigma, cfg.bias_value)
        self._check_cfl()

    def _initialize(self, init, random_amp, blob_sigma, bias_value):
        if init == "random":
            self.phi[:] = random_amp * (self.rng.random((self.nx, self.ny)) - 0.5)
        elif init == "blob":
            i = np.arange(self.nx); j = np.arange(self.ny)
            ii, jj = np.meshgrid(i, j, indexing="ij")
            r2 = (ii - (self.nx-1)/2.0)**2 + (jj - (self.ny-1)/2.0)**2
            self.phi[:] = np.exp(-r2/(2.0*blob_sigma*blob_sigma))
        elif init == "bias":
            self.phi[:] = bias_value
        else:
            raise ValueError("Unknown init")
        self.phi_prev[:] = self.phi

    def _check_cfl(self):
        s = self.c * self.cfg.dt / self.cfg.dx
        smax = 1.0 / math.sqrt(2.0)
        print(f"[INFO] CFL check: c*dt/dx = {s:.3f} (≤ {smax:.3f} recommended)")

    def step(self):
        # Per-cell λ, damping and noise
        lam_map = np.fromiter((self.genes[i,j].lam for i in range(self.nx) for j in range(self.ny)),
                              dtype=self.dtype).reshape(self.nx, self.ny)
        damp_map = np.fromiter((self.genes[i,j].damping for i in range(self.nx) for j in range(self.ny)),
                               dtype=self.dtype).reshape(self.nx, self.ny)
        noise_map = np.fromiter((self.genes[i,j].noise_amp for i in range(self.nx) for j in range(self.ny)),
                                dtype=self.dtype).reshape(self.nx, self.ny)

        lap = laplacian_5pt(self.phi, self.dx)
        force = (self.c*self.c) * lap - (lam_map * self.phi * (self.phi*self.phi - self.v*self.v))

        # Add per-cell noise as acceleration
        if np.any(noise_map > 0):
            force += noise_map * self.rng.standard_normal(self.phi.shape).astype(self.dtype)

        damp = (self.eta * damp_map) * self.dt
        phi_new = (2.0 - damp) * self.phi - (1.0 - damp) * self.phi_prev + (self.dt*self.dt) * force

        self.phi_prev, self.phi = self.phi, phi_new

        # Global perturbation
        self.apply_perturbation(self.cfg.perturb_mag, self.cfg.perturb_prob)

        # Mutate genes (slowly)
        for i in range(self.nx):
            for j in range(self.ny):
                self.genes[i,j].mutate(self.rng)

    def apply_perturbation(self, mag, prob):
        if mag <= 0 or prob <= 0:
            return
        mask = self.rng.random((self.nx, self.ny)) < prob
        kick = mag * (2.0 * self.rng.random((self.nx, self.ny)) - 1.0)
        self.phi += mask * kick

# =========================
# SubFields (agents)
# =========================
@dataclass
class SubField:
    x0: int
    y0: int
    size: int
    last_score: float = 0.0

    def window(self, field: Field):
        xs = slice(self.x0, self.x0 + self.size)
        ys = slice(self.y0, self.y0 + self.size)
        return xs, ys

    def send_signal(self, psi: np.ndarray, amp=0.05):
        xs, ys = self.window(None)
        cx = self.x0 + self.size//2
        cy = self.y0 + self.size//2
        # Gaussian packet
        X, Y = np.meshgrid(np.arange(psi.shape[0]), np.arange(psi.shape[1]), indexing="ij")
        gauss = amp * np.exp(-((X - cx)**2 + (Y - cy)**2)/(2*(self.size/3)**2))
        psi += gauss

    def receive_signal(self, psi: np.ndarray):
        xs, ys = self.window(None)
        patch = psi[xs, ys]
        return float(patch.mean())

    def simulate_local_future(self, field: Field, steps: int, action_kick: float, rng) -> float:
        xs, ys = self.window(field)
        local_phi = field.phi[xs, ys].copy()
        local_prev = field.phi_prev[xs, ys].copy()
        # Tiny copy of dynamics: homogeneous genes approximated by local averages
        lam_loc = np.mean([[field.genes[i,j].lam for j in range(ys.start, ys.stop)]
                           for i in range(xs.start, xs.stop)])
        damp_loc = np.mean([[field.genes[i,j].damping for j in range(ys.start, ys.stop)]
                            for i in range(xs.start, xs.stop)])
        noise_loc = np.mean([[field.genes[i,j].noise_amp for j in range(ys.start, ys.stop)]
                             for i in range(xs.start, xs.stop)])
        # Apply a random action
        local_phi += action_kick * (2.0 * rng.random(local_phi.shape) - 1.0)

        def step_local(phi, phi_prev):
            lap = laplacian_5pt(phi, field.dx)
            force = (field.c*field.c) * lap - (lam_loc * phi * (phi*phi - field.v*field.v))
            force += noise_loc * rng.standard_normal(phi.shape)
            damp = (field.eta * damp_loc) * field.dt
            phi_new = (2.0 - damp) * phi - (1.0 - damp) * phi_prev + (field.dt*field.dt)*force
            return phi_new, phi

        # Roll out
        for _ in range(steps):
            local_phi, local_prev = step_local(local_phi, local_prev)

        # Score: encourage low potential + low gradient (calm) and low variance (coherence)
        gx, gy = gradients_central(local_phi, field.dx)
        Ep = V_phi4(local_phi, lam_loc, field.v)
        score = - (Ep.mean() + 0.2 * (gx*gx + gy*gy).mean() + 0.05 * local_phi.var())
        return float(score)

# =========================
# Dream manager (selection + mutation of physics)
# =========================
class DreamManager:
    def __init__(self, cfg: Config):
        self.cfg = cfg

    def mutate_physics(self, cfg: Config, rng) -> Config:
        new = Config(**asdict(cfg))
        # soft jitter of lam, eta, random_amp
        new.lam = max(0.1, cfg.lam + rng.normal(0, 0.05))
        new.eta = max(0.0, cfg.eta + rng.normal(0, 0.005))
        new.random_amp = max(0.0, cfg.random_amp + rng.normal(0, 0.005))
        return new

    def select_worlds(self, worlds: List[Tuple[Config, float]], k: int) -> List[Tuple[Config, float]]:
        worlds_sorted = sorted(worlds, key=lambda x: x[1], reverse=True)
        return worlds_sorted[:k]

# =========================
# Runner
# =========================
def plot_field(phi, step, out_dir, cmap):
    ensure_dir(out_dir)
    plt.figure(figsize=(6,6))
    im = plt.imshow(phi.T, cmap=cmap, origin="lower")
    plt.colorbar(im, fraction=0.046, pad=0.04, label="φ")
    plt.title(f"Field at step {step}")
    plt.tight_layout()
    plt.savefig(os.path.join(out_dir, f"field_{step:06d}.png"), dpi=120)
    plt.close()

def plot_structures(phi, structures, step, out_dir, cmap):
    ensure_dir(out_dir)
    plt.figure(figsize=(6,6))
    plt.imshow(phi.T, cmap=cmap, origin="lower")
    if len(structures):
        # swap coordinates for display (x,y) -> (col,row)
        pts = np.array([[j,i] for i,j in structures])
        plt.scatter(pts[:,0], pts[:,1], s=60, facecolors="none", edgecolors="yellow", linewidths=1.5)
    plt.title(f"Detected structures at step {step}")
    plt.tight_layout()
    plt.savefig(os.path.join(out_dir, f"struct_{step:06d}.png"), dpi=120)
    plt.close()

def run_world(cfg: Config, rng: np.random.Generator, out_dir: str) -> float:
    ensure_dir(out_dir)
    save_json(os.path.join(out_dir, "config.json"), asdict(cfg))
    field = Field(cfg, rng)
    psi = np.zeros_like(field.phi)  # signal field

    # SubFields placed on a grid
    subs: List[SubField] = []
    step_x = max(1, (cfg.nx - cfg.sub_size) // int(math.sqrt(cfg.n_subfields)))
    step_y = max(1, (cfg.ny - cfg.sub_size) // int(math.sqrt(cfg.n_subfields)))
    for ix in range(0, cfg.nx - cfg.sub_size + 1, step_x):
        for iy in range(0, cfg.ny - cfg.sub_size + 1, step_y):
            if len(subs) < cfg.n_subfields:
                subs.append(SubField(ix, iy, cfg.sub_size))

    energy_csv = os.path.join(out_dir, "energy.csv")
    with open(energy_csv, "w", newline="") as f:
        w = csv.writer(f); w.writerow(["step","E","n_struct","avg_sub_score"])

    best_avg_score = -1e9
    for step in range(cfg.steps + 1):
        # Structures
        if step % cfg.save_every == 0:
            structs, Es = detect_structures(field.phi, cfg.dx, cfg.lam, cfg.v,
                                            sigma=cfg.struct_sigma, win=cfg.struct_win, thresh=cfg.struct_thresh)
            plot_field(field.phi, step, os.path.join(out_dir, "frames"), cfg.cmap)
            plot_structures(field.phi, structs, step, os.path.join(out_dir, "structs"), cfg.cmap)

            # Subfields: plan and act
            sub_scores = []
            for sf in subs:
                # receive and send signals
                s_val = sf.receive_signal(psi)
                sf.send_signal(psi, amp=0.02 + 0.02*abs(s_val))
                # try a few actions and pick best local plan
                scores = [sf.simulate_local_future(field, cfg.plan_steps, cfg.action_kick, rng)
                          for _ in range(cfg.actions_per_step)]
                sf.last_score = float(np.max(scores))
                sub_scores.append(sf.last_score)

            avg_score = float(np.mean(sub_scores)) if sub_scores else 0.0
            E = total_energy(field.phi, field.phi_prev, cfg.dt, cfg.dx, cfg.c, cfg.lam, cfg.v)
            with open(energy_csv, "a", newline="") as f:
                w = csv.writer(f); w.writerow([step, E, len(structs), avg_score])

            if avg_score > best_avg_score:
                best_avg_score = avg_score

        # Advance physics and dissipate psi
        field.step()
        psi *= 0.98  # signal decay

    return best_avg_score

def evolve_dream_levels(cfg: Config):
    base_rng = np.random.default_rng(cfg.seed)
    manager = DreamManager(cfg)

    worlds = [(cfg, 0.0)]
    for level in range(cfg.dream_levels):
        next_worlds = []
        for wcfg, _ in worlds:
            # spawn a few mutated worlds
            candidates = []
            for k in range(4):
                child = manager.mutate_physics(wcfg, base_rng)
                tag = f"level{level}_cand{k}"
                score = run_world(child, base_rng, os.path.join(cfg.out_dir, tag))
                candidates.append((child, score))
            # select survivors
            survivors = manager.select_worlds(candidates, cfg.survivors_per_level)
            for scfg, sscore in survivors:
                next_worlds.append((scfg, sscore))
        worlds = next_worlds
        print(f"[LEVEL {level}] survivors:", [round(s,3) for _, s in worlds])
    return worlds

# =========================
# CLI
# =========================
def build_parser():
    p = argparse.ArgumentParser(description="Dreamstack: field → structures → agents → dreams")
    p.add_argument("--out_dir", type=str, default="out_dreamstack")
    p.add_argument("--steps", type=int, default=1500)
    p.add_argument("--nx", type=int, default=192)
    p.add_argument("--ny", type=int, default=192)
    p.add_argument("--save_every", type=int, default=50)
    p.add_argument("--seed", type=int, default=42)
    p.add_argument("--dream_levels", type=int, default=2)
    p.add_argument("--survivors_per_level", type=int, default=2)
    return p

def main(argv=None):
    parser = build_parser()
    args, _ = parser.parse_known_args(argv)

    cfg = Config(out_dir=args.out_dir, steps=args.steps, nx=args.nx, ny=args.ny,
                 save_every=args.save_every, seed=args.seed,
                 dream_levels=args.dream_levels, survivors_per_level=args.survivors_per_level)

    t0 = time.time()
    survivors = evolve_dream_levels(cfg)
    dt = time.time() - t0
    print(f"[DONE] Dream evolution finished in {dt/60:.2f} min")
    # Persist survivors
    out = os.path.join(cfg.out_dir, "survivors.json")
    save_json(out, [asdict(c) | {"score": s} for c, s in survivors])
    print(f"[SAVED] Survivors -> {out}")

if __name__ == "__main__":
    main()